#Comparison of Mistral/ LLama-2 / Falcon / MPT models - assess similarity of movies recommendations

We're going to compare a couple of most prominent open source LLMs on the task of assessing if two movie recommendation requests are similar or not. Llama-2 has been praised as achieving high level of general language understanding
*MMLU* (the benchmark measures a text model’s multitask accuracy), but reports on a par with MPT and Falcon on tasks like:
- *HellaSwag* (commonsense inference),
- *BoolQA* (question-answering dataset for yes/no questions) or
- *WinoGrande* (where the goal is to choose the right option for a given sentence that requires commonsense reasoning).
Check latest comparison from [Meta's benchmark results for Llama-2 and other models here](https://ai.meta.com/resources/models-and-libraries/llama/).

Recently released Mistral 7B reportedly outperforms the best open  13B model (Llama 2) across all evaluated benchmarks, and the best released 34B model (Llama 1) in reasoning, mathematics, and code generation.

We're going to compare capabilities of the 4 small (7B) models and also try out Llama-2-13B to see if it improves on the results. All models are going to be compared in *zero-shot* manner. Additionally, Mistral and Llama are going to be tested in *few-shot* manner as well.

In [1]:
!pip install -qU transformers accelerate einops xformers bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 112.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.6 MB/s eta 0:00:00


### Get data

In [5]:
import pickle

with open('/content/drive/MyDrive/ML/data/movies/reddit/bert_final_dataset.pkl', 'rb') as f:
    data = pickle.load(f)
    training_pairs = data['training_pairs']
    val_set = data['val_set']
len(training_pairs), len(val_set)

(55882, 1816)

Let's see an example of positive matching between two movie requests recommendations, as well as a negative example.

POSITIVE:

In [ ]:
training_pairs[101]['query'], training_pairs[101]['submission']

('Any movie that has elements of gambling in it or based on it. Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?',
 "Fun movies taking place in a casino?I recently watched the 2006 Casino Royale, and really loved the middle ~45 minutes in which it is a glitzy thriller of simply trying to win a card game surrounded by decadence. It's an itch I want to scratch, but pretty much every movie about gambling tries to push the (albeit correct) message that gambling is a social blight, harmful, addictive, exploitative, etc.\nAre there any movies that just *have fun* with the gaudy excess of it all? Maybe movies that simply use the casino as a setting rather than a theme?\nThanks!")

NEGATIVE:

In [ ]:
training_pairs[102]['query'], training_pairs[102]['submission']

('Any movie that has elements of gambling in it or based on it. Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?',
 'Looking for a serious movie about gambling addiction. Hi all - without naming names, too many movies about gambling involve the main character putting in one last bet and winning it all and having a happy ending. Does anyone have any recommendations for movies that look at gambling addiction in a much more realistic, darker view?')

In [6]:
standard_prompts =  ['''There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: ''',
     '''There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: ''']

import datasets
import pandas as pd

def get_query(sample):
    return f"""There are two movie recommendation requests from two users:
REQUEST 1: "{sample['query']}"

REQUEST 2: "{sample['submission']}"

Answer the following yes/no question.
QUESTION: Can these two users be recommended similar titles?

ANSWER:"""

SAMPLE_SIZE = 88

def get_datapoint(sample):
    q = get_query(sample)
    return {'text': q}

val_set_queries = list(map(get_datapoint, val_set[:SAMPLE_SIZE]))

val_ds = datasets.Dataset.from_pandas(pd.DataFrame(data = val_set_queries))
print(val_set_queries[0]['text']), val_ds

There are two movie recommendation requests from two users:
REQUEST 1: "Movies exactly like Fantastic Mr Ripley and Match Point?Looking for movies where the main character is an impostor/liar/con man who cheats, lies, manipulates his way into a wealthy family. Exactly like that.
 
(Please do not suggest Parasite cause it's not the same movie. Once in every 24 hours, somebody recommends Parasite in this sub even if it's an unrelated movie)"

REQUEST 2: "Movies where the protagonist is a con artist. Suggest me some movies where the protagonist is a con artist. Movies like Catch me if you can, Matchstick Men, The Wolf of Wall Street, The Imposter, etc. Thanks a lot!!"

Answer the following yes/no question.
QUESTION: Can these two users be recommended similar titles?

ANSWER:


(None,
 Dataset({
     features: ['text'],
     num_rows: 88
 }))

Prepare also some few-shot prompts for comparison:

In [13]:
def get_incontext_query(sample):
    return f'''Example1: There are two movie recommendation requests from two users:
REQUEST 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
REQUEST 2: "Please recommend me some good action-packed or war movies. No romances pls!"

Answer the following yes/no question in a precise and short style.

QUESTION: Can these two users be recommended similar titles?
ANSWER: No


Example 2: There are two movie recommendation requests from two users:
REQUEST 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
REQUEST 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"

Answer the following yes/no question in a precise and short style.

QUESTION: Can these two users be recommended similar titles?
ANSWER: Yes


Example 3: There are two movie recommendation requests from two users:
REQUEST 1: "{sample['query']}"
REQUEST 2: "{sample['submission']}"

Answer the following yes/no question.

QUESTION: Can these two users be recommended similar titles?
ANSWER: '''


In [7]:
def get_incontext_query2(sample, open_tag='', close_tag=''):
    return f'''{open_tag} EXAMPLE 1: There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Any movie that has elements of gambling in it or based on it.
Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?"

REQUEST 2: "Looking for a serious movie about gambling addiction.
Too many movies about gambling involve the main character putting in one last bet and winning it all and having a happy ending.
Does anyone have any recommendations for movies that look at gambling addiction in a much more realistic, darker view?"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: No {close_tag}


{open_tag}EXAMPLE 2: There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Any movie that has elements of gambling in it or based on it.
Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?"

REQUEST 2: "Fun movies taking place in a casino?
I recently watched the 2006 Casino Royale, and really loved the middle ~45 minutes in which it is a glitzy thriller of simply trying to win a card game surrounded by decadence.
Pretty much every movie about gambling tries to push the (albeit correct) message that gambling is a social blight, harmful, addictive, exploitative, etc.
Are there any movies that just *have fun* with the gaudy excess of it all? Maybe movies that simply use the casino as a setting rather than a theme?"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: Yes{close_tag}


{open_tag}EXAMPLE 3: There are two movie recommendation REQUESTS from two users:

REQUEST 1: "{sample['query']}"

REQUEST 2: "{sample['submission']}"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: {close_tag}'''


In [15]:
def get_incontext_query3(sample):
    return f'''EXAMPLE 1: There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Post 2010 good movies to watch alone.
Many cinephiles tend to recommend older movies like Fight Club, American Psycho, 12 Angry Men etc. I like new movies like Good Time, The Lighthouse, 1917, Spree etc.
Recommend me some movies that I could watch while enjoying my day off. Basically something to make my day exciting."

REQUEST 2: "Stuck at home,suggest me a movie.So I am in isolation for a next few days.I am not looking any specific genres.
I prefer newer movies,comedy,mystery,thriller,horror.Some stuff I watched in recent time and liked:Brick,In Bruges, The lighthouse, Donnie Darko,Captian phillips,Oldboy.
I watched most of the most popular stuff."

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: Yes


EXAMPLE 2:  There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Post 2010 good movies to watch alone.
Many cinephiles tend to recommend older movies like Fight Club, American Psycho, 12 Angry Men etc. I like new movies like Good Time, The Lighthouse, 1917, Spree etc.
Recommend me some movies that I could watch while enjoying my day off. Basically something to make my day exciting."

REQUEST 2: "Lazy Saturday Afternoon Movie suggestions- Ideally where the protagonists get their shit together by the end(examples added).
Examples of the top of my head-that I have seen and liked are: Brittany Runs a Marathon or maybe Yes Man (with Jim Carrey).
Any other movie suggestions for a lazy Saturday regardless of the genre are also most welcome."

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER:  No


EXAMPLE 3: There are two movie recommendation REQUESTS from two users:

REQUEST 1: "{sample['query']}"

REQUEST 2: "{sample['submission']}"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: '''


In [ ]:
import datasets
import pandas as pd

SAMPLE_SIZE = 88

def get_datapoint(sample):
    q = get_incontext_query2(sample, open_tag='[INST]', close_tag='[/INST]')
    return {'text': q}

val_set_queries = list(map(get_datapoint, val_set[:SAMPLE_SIZE]))

val_ds = datasets.Dataset.from_pandas(pd.DataFrame(data = val_set_queries))
print(val_set_queries[0]['text']), val_ds

In [13]:
from transformers import AutoTokenizer
import transformers
import torch
from sklearn import metrics as mcs

def run_pipeline(sequences, pipeline, tokenizer):
    sequences = pipeline(sequences,
        eos_token_id=tokenizer.eos_token_id,
    )

    for seq in sequences:
        print(f"Result: {seq[0]['generated_text']}")

def run_inference(seq, results, answer_key = 'Answer:', remove_emoji = False):
    ans = seq[0]['generated_text']
    if remove_emoji:
        ans = remove_emojis(ans)

    ind = ans.rfind(answer_key)
    if ind != -1:
        ans = ans[ind+len(answer_key):]
        ans = ans.replace('[/INST]', '')
        ans = ans.strip().lower()
        print(ans)

        if ans.startswith('yes'):
            results.append(1)
        elif ans.startswith('no'):
            results.append(0)
        else:
            results.append(-1)
    else:
        results.append(-1)

def check_metrics(responses):
    y_true = [v['MATCH'] for v in val_set[:88]]
    print(mcs.classification_report(y_true, responses, digits = 4))

## 2. Mistral 7B-instruct

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
from torch import cuda, bfloat16

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'


tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model_name = 'mistralai/Mistral-7B-Instruct-v0.1'

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map='auto'
)

pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    do_sample=False,
    torch_dtype=torch.bfloat16,
    temperature=0.0,
    max_new_tokens=100,
    repetition_penalty=1.1,
    device_map="auto" # if you have GPU
)
model.eval()
print(f"Model loaded on {device}")

In [22]:
run_pipeline(standard_prompts, pipeline, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Result: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: 
No
Result: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: 
Yes


Looks promising - Mistral 7B can discern between different requests. Let's now run inference on a sample of 88 datapoints to be sure:

In [ ]:
%%time
from transformers.pipelines.pt_utils import KeyDataset
tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    eos_token_id=tokenizer.eos_token_id
)
responses = []
for seq in sequences:
    run_inference(seq, responses, answer_key = 'ANSWER:')
    if len(responses) % 10 == 0:
        print(responses)

print(responses)

In [26]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        39
           1     0.5568    1.0000    0.7153        49

    accuracy                         0.5568        88
   macro avg     0.2784    0.5000    0.3577        88
weighted avg     0.3100    0.5568    0.3983        88



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Looks like the model answered "Yes" to all queries. So the actual examples from evaluation set (coming from real Reddit examples) are too difficult/nuanced for Mistral-7B in zero-shot fashion.

Lets now try one of the prepared few-shot learning prompts:

In [ ]:
%%time
from transformers.pipelines.pt_utils import KeyDataset
tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    eos_token_id=tokenizer.eos_token_id
)
responses = []
for seq in sequences:
    run_inference(seq, responses, answer_key = 'ANSWER:')
    if len(responses) % 10 == 0:
        print(responses)

print(responses)

In [15]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     0.7143    0.6410    0.6757        39
           1     0.7358    0.7959    0.7647        49

    accuracy                         0.7273        88
   macro avg     0.7251    0.7185    0.7202        88
weighted avg     0.7263    0.7273    0.7252        88



Great result! With no fine tuning and just two examples (one positive and one negative) in the prompt itself Mistral improved its reasoning by a large margin!

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##2. Llama2-7B-instruct

In [ ]:
%%time
import torch
from torch import cuda, bfloat16
import transformers

model_name = "meta-llama/Llama-2-7b-chat-hf" #chat-hf (hugging face wrapper version)

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

hf_auth =  # '<LLAMA_HF_AUTH_TOKEN>'
model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    config=model_config,
    torch_dtype=torch.bfloat16,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

In [ ]:
%%time
import torch

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)

pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    do_sample=False,
    torch_dtype=torch.bfloat16,
    temperature=0.0,
    max_new_tokens=100,
    repetition_penalty=1.1,
    device_map="auto" # if you have GPU
)

In [ ]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


In [ ]:
run_pipeline(sequences, pipeline, tokenizer)

Result: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: 👍 Yes
Result: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: 👍 Yes


This doesn't look good. The model can't discern between clearly different requests...

Let's run inference on a sample of 88 datapoints to be sure:

In [ ]:
%%time
from transformers.pipelines.pt_utils import KeyDataset
tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    eos_token_id=tokenizer.eos_token_id
)
responses = []
for seq in sequences:
    run_inference(seq, responses, answer_key = 'Answer:', remove_emoji= True)
    if len(responses) % 10 == 0:
        print(responses)

print(responses)

In [ ]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000        39
           1     0.5568    1.0000    0.7153        49

    accuracy                         0.5568        88
   macro avg     0.2784    0.5000    0.3577        88
weighted avg     0.3100    0.5568    0.3983        88



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


As suspected, the models answers "YEs" to every question.

In [ ]:
print(val_ds[1]['text'])

Example1: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: No

Example 2: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"
Answer the following yes/no question in a precise and short style.
Question: Can these two users be recommended similar titles?
Answer: Yes

There are two movie recommendation requests from two users:
Request from User 1: "Movies exactly like Fantastic Mr Ripley and Match Point?Looking for movies where the main character is an impost

Try inference on a few-shot learning prompts:

In [ ]:
%%time
from transformers.pipelines.pt_utils import KeyDataset
tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    # batch_size=4,
    # max_length=100,
    eos_token_id=tokenizer.eos_token_id
)
responses = []
for seq in sequences:
    run_inference(seq, responses, answer_key = 'Answer:')
    if len(responses) % 10 == 0:
        print(responses)

print(responses)

In [ ]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     1.0000    0.1026    0.1860        39
           1     0.5833    1.0000    0.7368        49

    accuracy                         0.6023        88
   macro avg     0.7917    0.5513    0.4614        88
weighted avg     0.7680    0.6023    0.4927        88



With two examples given in the prompt the model was able to correctly recognize only 3 negative examples (out of 39). Results from Mistral are much more promising.

#### Run Llama-2-13B

Compare with bigger model. Load with quantization using bitsandbytes.

In [ ]:
%%time
from torch import cuda, bfloat16
import transformers

model_name = "meta-llama/Llama-2-13b-chat-hf"
# set quantization configuration using bitsandbytes lib
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)


hf_auth = # '<LLAMA_HF_AUTH_TOKEN>'
model_config = transformers.AutoConfig.from_pretrained(
    model_name,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()

In [ ]:
run_pipeline(standard_prompts, pipeline, tokenizer)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Result: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question.
Question: Can these two users be recommended similar titles?
Answer:  No. The first user is looking for lighthearted movies with no violence, while the second user wants action-packed or war movies with no romance. These two requests have conflicting preferences and cannot be recommended similar titles.
Result: There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"
Answer the following yes/no question.
Question: Can these two users be recommended similar titles?
Answer:  Yes. Both users are looking f

This looks very promising! This is the only model so far (compared to 7B versions of Llama-2, Falcon and MPT) where the model was able to discern between the two requests.
The requets, however, are pretty naive and the dataset contains more nuanced comaparisons. Let's see how Llama-13B does.

Let's try some more nuanced requests and see if Llama-13B can recognize the differences:

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch


sequences = pipeline(
    ['''There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Any movie that has elements of gambling in it or based on it. Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?"

REQUEST 2: "Looking for a serious movie about gambling addiction. Hi all - without naming names, too many movies about gambling involve the main character putting in one last bet and winning it all and having a happy ending. Does anyone have any recommendations for movies that look at gambling addiction in a much more realistic, darker view?"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: ''',
'''There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Any movie that has elements of gambling in it or based on it. Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?"

REQUEST 2: "Fun movies taking place in a casino?I recently watched the 2006 Casino Royale, and really loved the middle ~45 minutes in which it is a glitzy thriller of simply trying to win a card game surrounded by decadence. It's an itch I want to scratch, but pretty much every movie about gambling tries to push the (albeit correct) message that gambling is a social blight, harmful, addictive, exploitative, etc.\nAre there any movies that just *have fun* with the gaudy excess of it all? Maybe movies that simply use the casino as a setting rather than a theme?\nThanks!"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER: '''],
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Result: There are two movie recommendation REQUESTS from two users:

REQUEST 1: "Any movie that has elements of gambling in it or based on it. Just watched the hangover (ikr) and the scene were there was a montage of alan was gambling made me wonder if there are any films that is based on that or has elements of it like casino royale. Any suggestions?"

REQUEST 2: "Looking for a serious movie about gambling addiction. Hi all - without naming names, too many movies about gambling involve the main character putting in one last bet and winning it all and having a happy ending. Does anyone have any recommendations for movies that look at gambling addiction in a much more realistic, darker view?"

Answer the following yes/no question.
Question: Can these two users be recommended similar titles?

ANSWER:  No. The first user is looking for movies with elements of gambling, while the second user is looking for a serious movie about gambling addiction. These requests do not align well, as the f

Cool! This is still good results from Llama 13B!

In [ ]:
%%time
from tqdm import tqdm
from transformers.pipelines.pt_utils import KeyDataset

tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    # batch_size=4,
    temperature=None,
    top_p=0.0,
    eos_token_id=tokenizer.eos_token_id
)

responses = []
# for i in tqdm(range(len(val_ds))):
for seq in sequences:
    run_inference(seq, responses, answer_key = 'ANSWER:')
    if len(responses) % 10 == 0:
        print(responses)

print(responses)

Results for few-shot task:

In [ ]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     1.0000    0.0513    0.0976        39
           1     0.5698    1.0000    0.7259        49

    accuracy                         0.5795        88
   macro avg     0.7849    0.5256    0.4117        88
weighted avg     0.7604    0.5795    0.4474        88



The results are even worse. The reason is the used prompt was encouraging the model say "yes" to the example where it gave otherwise "no" answer.
TODO: More complex "no" type of prompts need to be prepared and tried.

Results for zero-shot task:

In [ ]:
from sklearn import metrics as mcs
y_true = [v['MATCH'] for v in val_set[:SAMPLE_SIZE]]
print(mcs.classification_report(y_true, responses, digits = 4))

              precision    recall  f1-score   support

           0     1.0000    0.0769    0.1429        39
           1     0.5765    1.0000    0.7313        49

    accuracy                         0.5909        88
   macro avg     0.7882    0.5385    0.4371        88
weighted avg     0.7642    0.5909    0.4705        88



Llama-2-13B gave predominantly "Yes" answers. The two "No" answers in the result set were correct, but from looking at the produced answers it seems like the model was trying to find far-fetched similarities between the requests.

Several prompts were tried and overall performance was disappointing. It seems that the questions are too nuanced for the 13B model. Bigger model or *in-context learning* could be tried going further.

So far Mistral-7B seems to beat Llama2, even the 13B model!

##1. Falcon-7B-instruct

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines.pt_utils import KeyDataset
import transformers
import torch

def init_model():
    model_name = "tiiuae/falcon-7b-instruct"

    tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
    pipeline = transformers.pipeline(
        "text-generation",
        model=model_name,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        # batch_size=8
    )
    return pipeline, tokenizer


pipeline, tokenizer = init_model()


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
pipeline.model.config.pad_token_id

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    batch_size=4,
    max_length=255,
    do_sample=False,
    # top_p = 0.25,
    temperature = 0,
    # top_k=0,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)
responses = []
for seq in sequences:
   run_inference(seq, responses, answer_key = 'Answer:')

print(responses)


In [ ]:
len(responses)

88

In [ ]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     0.4706    0.4103    0.4384        39
           1     0.5741    0.6327    0.6019        49

    accuracy                         0.5341        88
   macro avg     0.5223    0.5215    0.5201        88
weighted avg     0.5282    0.5341    0.5294        88



Cross check with results from HuggingFace inference endpoint:

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/tiiuae/falcon-7b-instruct"
API_TOKEN = #"<YOUR_HF_KEY_HERE>"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output1 = query({
	"inputs": ''''There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"
Answer the following yes/no question.
Question: Can these two users be recommended similar titles?
Answer: ''',
})
output2 = query({
	"inputs": '''There are two movie recommendation requests from two users:
Request 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question.
Question: Can these two users be recommended similar titles?
Answer: ''',
})
output1, output2

([{'generated_text': '\'There are two movie recommendation requests from two users:\nRequest 1: "Lighthearted movies, about love or friendship, I don\'t want it to contain any violence."\nRequest 2: "Please recommend me some funny romcom or a feel good movie to watch with my gf!"\nAnswer the following yes/no question.\nQuestion: Can these two users be recommended similar titles?\nAnswer: "Yes"'}],
 [{'generated_text': 'There are two movie recommendation requests from two users:\nRequest 1: "Lighthearted movies, about love or friendship, I don\'t want it to contain any violence."\nRequest 2: "Please recommend me some good action-packed or war movies. No romances pls!"\nAnswer the following yes/no question.\nQuestion: Can these two users be recommended similar titles?\nAnswer: "Yes"'}])

Falcon-7B is unable to discern between two different requests.

### Results:
On small sample of 88 examples it seems that the small Falcon model does not comprehend the text and is not able to differentiate between two different movie request recommendations.

##2. MPT-7B-instruct

The model was trained on data formatted in the dolly-15k format, like below:

In [ ]:
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
PROMPT_FOR_GENERATION_FORMAT = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
)

example = '''There are two movie recommendation requests from two users:
Request from User 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request from User 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question. Can these two users be recommended similar titles?
'''
fmt_ex = PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)
print(fmt_ex)

Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
There are two movie recommendation requests from two users:
Request from User 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request from User 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question. Can these two users be recommended similar titles?

### Response:



In [ ]:
import datasets
import pandas as pd

SAMPLE_NUM = 88 #1816

def get_query(sample):
    example = f'''There are two movie recommendation requests from two users:
Request from User 1: "{sample['query']}"
Request from User 2: "{sample['submission']}"
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer:'''
    return example #PROMPT_FOR_GENERATION_FORMAT.format(instruction=example)

def get_datapoint(sample):
    q = get_query(sample)
    return {'text': q}

val_set_queries = list(map(get_datapoint, val_set[:SAMPLE_NUM]))
val_ds = datasets.Dataset.from_pandas(pd.DataFrame(data = val_set_queries))
print(val_set_queries[0]['text']), val_ds

There are two movie recommendation requests from two users:
Request from User 1: "Movies exactly like Fantastic Mr Ripley and Match Point?Looking for movies where the main character is an impostor/liar/con man who cheats, lies, manipulates his way into a wealthy family. Exactly like that.
 
(Please do not suggest Parasite cause it's not the same movie. Once in every 24 hours, somebody recommends Parasite in this sub even if it's an unrelated movie)"
Request from User 2: "Movies where the protagonist is a con artist. Suggest me some movies where the protagonist is a con artist. Movies like Catch me if you can, Matchstick Men, The Wolf of Wall Street, The Imposter, etc. Thanks a lot!!"
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer:


(None,
 Dataset({
     features: ['text'],
     num_rows: 88
 }))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.pipelines.pt_utils import KeyDataset
import transformers
import torch

def init_model():
    model_name = "mosaicml/mpt-7b-instruct"
    config = transformers.AutoConfig.from_pretrained(model_name, trust_remote_code=True)
    config.attn_config['attn_impl'] = 'triton'
    config.init_device = 'cuda:0' # For fast initialization directly on GPU!


    tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

    pipeline = transformers.pipeline(
        "text-generation",
        model=model_name,
        tokenizer=tokenizer,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        device_map="auto",
        # batch_size=8
    )
    return pipeline, tokenizer


pipeline, tokenizer = init_model()

In [ ]:
tokenizer.pad_token_id = tokenizer.eos_token_id

sequences = pipeline(KeyDataset(val_ds, 'text'),
    # batch_size=4,
    max_length=100,
    do_sample=False,
    # top_p = 0.25,
    temperature = 0,
    # top_k=0,
    num_return_sequences=1,
    use_cache=True,
    eos_token_id=tokenizer.eos_token_id
)

responses = []
for seq in sequences:
    run_inference(seq, responses, answer_key = 'Answer:')

print(responses)


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


There are two movie recommendation requests from two users:
Request from User 1: "Movies exactly like Fantastic Mr Ripley and Match Point?Looking for movies where the main character is an impostor/liar/con man who cheats, lies, manipulates his way into a wealthy family. Exactly like that.
 
(Please do not suggest Parasite cause it's not the same movie. Once in every 24 hours, somebody recommends Parasite in this sub even if it's an unrelated movie)"
Request from User 2: "Movies where the protagonist is a con artist. Suggest me some movies where the protagonist is a con artist. Movies like Catch me if you can, Matchstick Men, The Wolf of Wall Street, The Imposter, etc. Thanks a lot!!"
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer: No
 No


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


There are two movie recommendation requests from two users:
Request from User 1: "Movies exactly like Fantastic Mr Ripley and Match Point?Looking for movies where the main character is an impostor/liar/con man who cheats, lies, manipulates his way into a wealthy family. Exactly like that.
 
(Please do not suggest Parasite cause it's not the same movie. Once in every 24 hours, somebody recommends Parasite in this sub even if it's an unrelated movie)"
Request from User 2: "Looking for movies where main character is the victim of an elaborate ruse, where everyone they know is eventually revealed to have been an actor after they’ve been fooled/manipulated/entrapped.Bonus points if the main character’s reputation is ruined as a result of the actions they took under these false pretenses."
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer: No
 No


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...

There are two movie recommendation requests from two users:
Request from User 1: "What are your 5 best Werner Herzog Movies?Are any of you familiar with Werner Herzog Movies? I'm writing a blog related to his most notable movies. Let me know your best 5"
Request from User 2: "Werner Herzog Documentaries. I love his scripted films, but the only documentary of his I’ve seen is Grizzly Man. What Herzog documentary should I watch next?"
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer: No
 No


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 101, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


There are two movie recommendation requests from two users:
Request from User 1: "What are your 5 best Werner Herzog Movies?Are any of you familiar with Werner Herzog Movies? I'm writing a blog related to his most notable movies. Let me know your best 5"
Request from User 2: "your top 5 feature length experimental/avant-garde films?"
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer: Yes
 Yes


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 203, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


There are two movie recommendation requests from two users:
Request from User 1: "Funny heist or crime investigation movies. I am looking for movies similar to Logan Lucky, Nice guys, kiss kiss bang bang, Baby driver etc. Thrillers that also have some comedic element."
Request from User 2: "Comedic detective movies?Some of my favorite movies are The Big Lebowski, Kiss Kiss Bang Bang, and The Nice Guys. 
I like that these movies are sillier than a regular action or mystery movie. The main characters are all rough and odd and just kind of fall into solving a mystery. There are elaborate scenes like over the top LA parties and interesting costumes. I also like it when movies are set in a specific time period, like the Nice Guys was set in the 70’s. 
I’d love to hear your recommendations!"
Answer the following yes/no question. Can these two users be recommended similar titles?
Answer: No
 No
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

From the output it's clear the small model is producing mostly random results with domination of "No". The results for a small sample for dolly-15k format:

In [ ]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     0.4643    1.0000    0.6341        39
           1     1.0000    0.0816    0.1509        49

    accuracy                         0.4886        88
   macro avg     0.7321    0.5408    0.3925        88
weighted avg     0.7626    0.4886    0.3651        88



We also tried a different format of prompt - more standard kind without the '###Instruction' etc cues. The results were even worse:

In [ ]:
check_metrics(responses)

              precision    recall  f1-score   support

           0     0.4286    0.9231    0.5854        39
           1     0.2500    0.0204    0.0377        49

    accuracy                         0.4205        88
   macro avg     0.3393    0.4717    0.3116        88
weighted avg     0.3291    0.4205    0.2804        88



In [ ]:
sequences1 = pipeline(
    '''There are two movie recommendation requests from two users:
Request from User 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request from User 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question. Can these two users be recommended similar movies? Explain your answer.
Answer:
''',
    max_length=200,
    do_sample=False,
    top_k=0,
    temperature = 0,
    num_return_sequences=1,
    # eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences1:
    print(f"Result: {seq['generated_text']}")

Result: There are two movie recommendation requests from two users:
Request from User 1: "Lighthearted movies, about love or friendship, I don't want it to contain any violence."
Request from User 2: "Please recommend me some good action-packed or war movies. No romances pls!"
Answer the following yes/no question. Can these two users be recommended similar movies? Explain your answer.
Answer:
Yes, they can be recommended similar movies. The first user's request can be satisfied by the following movies:
"The Notebook"
"The Fault in Our Stars"
"About Time"
"The Perks of Being a Wallflower"
"The Best Exotic Marigold Hotel"
"The Big Sick"
"The Fault in Our Stars" is a romance movie, but it is a good fit for the first user's request. The second user's request can be satisfied by the following movies:



Result: MTP-7B is clearly too small for the task of comparing two movie recommendations.


### Summary

None of the 7B models was capable enough to discern between the nuances of recommendation requests.


From the tried 7B models on a small sample of 88 examples the performance results are as follow:
1. Mistral-7B-Instruct shows very promising results with few-shot learning prompts (one negative and one positive examples)
2. Llama2-7B AND Falcon-7B have on par results for the small sample. Llama 7B produced predominantly "yes" answer to every query, while Falcon produced both "yes" and "no" but with a quite a lot of randomness.
3. MPT-7B produced mostly random answers.

TODO: Try few-shot learning prompts also for the rest of models.
